<a href="https://colab.research.google.com/github/p09323028/2021s_NTU_Econometrics_II/blob/main/CH12_cig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chapter 12: Instrumental Variables Regression**
前置作業:
- import 套件
- 載入資料
- 讀取資料
- 資料處理(設置變數)

In [ ]:
!pip install linearmodels

In [ ]:
import pandas as pd
import numpy as np
import linearmodels.iv as iv
import statsmodels.formula.api as smf

In [ ]:
!gdown --id '1KQX9QBPIIn0XPXKoHpVBgTMlXG9iV_o0' --output cigarette.xlsx
!ls

Downloading...
From: https://drive.google.com/uc?id=1KQX9QBPIIn0XPXKoHpVBgTMlXG9iV_o0
To: /content/cigarette.xlsx
100% 12.6k/12.6k [00:00<00:00, 8.94MB/s]
cigarette.xlsx	sample_data


In [ ]:
cig = pd.read_excel('cigarette.xlsx')

In [ ]:
cig = cig.sort_values(['state', 'year'])
cig.head()

,state,year,cpi,pop,packpc,income,tax,avgprs,taxs
0,AL,1985,1.076,3973000,116.486282,46014968,32.500004,102.181671,33.348335
1,AR,1985,1.076,2327000,128.534592,26210736,37.000000,101.474998,37.000000
2,AZ,1985,1.076,3184000,104.522614,43956936,31.000000,108.578751,36.170418
3,CA,1985,1.076,26444000,100.363037,447102816,26.000000,107.837341,32.104000
4,CO,1985,1.076,3209000,112.963539,49466672,31.000000,94.266663,31.000000


In [ ]:
cig.columns

Index(['state', 'year', 'cpi', 'pop', 'packpc', 'income', 'tax', 'avgprs',
       'taxs'],
      dtype='object')

In [ ]:
# real average price during fiscal year, including sales taxes
cig['ravgprs'] = cig['avgprs'] / cig['cpi']

# real average Cig specifice tax during fiscal year
cig['rtax'] = cig['tax'] / cig['cpi']

# real average total tax during fiscal year,including sales taxes
cig['rtaxs'] = cig['taxs'] / cig['cpi']

# real average sales tax per pack during fiscal year
cig['rtaxso'] = cig['rtaxs'] - cig['rtax']

cig['lpackpc'] = np.log(cig['packpc'])

cig['lravgprs'] = np.log(cig['ravgprs'])

In [ ]:
cig['perinc'] = cig['income'] / (cig['pop'] * cig['cpi'])

cig['lperinc'] = np.log(cig['perinc'])

state_dummy = pd.get_dummies(cig['state'])

cig = pd.concat([cig, state_dummy], axis=1)

In [ ]:
cig.columns

Index(['state', 'year', 'cpi', 'pop', 'packpc', 'income', 'tax', 'avgprs',
       'taxs', 'ravgprs', 'rtax', 'rtaxs', 'rtaxso', 'lpackpc', 'lravgprs',
       'perinc', 'lperinc', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL',
       'GA', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI',
       'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY',
       'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT',
       'WA', 'WI', 'WV', 'WY'],
      dtype='object')

In [ ]:
cig['ltpackpc'] = np.log(cig['packpc'] / cig['packpc'].shift(1))
cig['ltavgprs'] = np.log(cig['ravgprs'] / cig['ravgprs'].shift(1))
cig['ltperinc'] = np.log(cig['perinc'] / cig['perinc'].shift(1))
cig['dtrtaxs'] = cig['rtaxs'] - cig['rtaxs'].shift(1)
cig['dtrtax'] = cig['rtax'] - cig['rtax'].shift(1)
cig['dtrtaxso'] = cig['rtaxso'] - cig['rtaxso'].shift(1)

In [ ]:
cig_1995 = cig[cig['year'] == 1995]

In [ ]:
cig_1995.head()

,state,year,cpi,pop,packpc,income,tax,avgprs,taxs,ravgprs,rtax,rtaxs,rtaxso,lpackpc,lravgprs,perinc,lperinc,AL,AR,AZ,CA,CO,CT,DE,FL,GA,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,ltpackpc,ltavgprs,ltperinc,dtrtaxs,dtrtax,dtrtaxso
48,AL,1995,1.524,4262731,101.085434,83903280,40.500004,158.371338,41.904671,103.918206,26.574807,27.496504,0.921697,4.615966,4.643604,12.915347,2.558416,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.141807,0.090102,0.182221,-3.496373,-3.629658,0.133285
49,AR,1995,1.524,2480121,111.042969,45995496,55.500000,175.542511,63.859169,115.185380,36.417325,41.902344,5.485019,4.709917,4.746543,12.169073,2.498898,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.146281,0.199981,0.150559,7.515726,2.030707,5.485019
50,AZ,1995,1.524,4306908,71.954170,88870496,65.333328,198.607498,74.790825,130.319887,42.869640,49.075347,6.205707,4.276029,4.869992,13.539638,2.605622,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.373374,0.255767,0.053800,15.459716,14.059230,1.400486
51,CA,1995,1.524,31493524,56.859306,771470144,61.000000,210.504669,74.771332,138.126430,40.026249,49.062556,9.036307,4.040580,4.928169,16.073591,2.777178,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.568214,0.320796,0.022669,19.226124,15.862679,3.363445
52,CO,1995,1.524,3738061,82.582924,92946544,44.000000,167.350006,44.000000,109.809720,28.871392,28.871392,0.000000,4.413803,4.698749,16.315557,2.792119,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.313262,0.225872,0.130030,0.060983,0.060983,0.000000


## **12.1 The IV Estimator with a Single Regressor and a Single Instrument**

### **Equation 12.9**
$\widehat{\text{ln}(P^{cig}_i)} = 4.62 + 0.031Salestax.$

In [ ]:
reg_12_9 = smf.ols(formula='lravgprs ~ rtaxso', data=cig_1995)
result_12_9 = reg_12_9.fit(cov_type='HC1')
print(result_12_9.summary())

                            OLS Regression Results                            
Dep. Variable:               lravgprs   R-squared:                       0.471
Model:                            OLS   Adj. R-squared:                  0.459
Method:                 Least Squares   F-statistic:                     40.39
Date:                Sun, 21 Mar 2021   Prob (F-statistic):           8.49e-08
Time:                        08:14:42   Log-Likelihood:                 46.435
No. Observations:                  48   AIC:                            -88.87
Df Residuals:                      46   BIC:                            -85.13
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.6165      0.029    159.644      0.0

In [ ]:
# Not Report
reg_n = smf.ols(formula='lpackpc ~ lravgprs', data=cig_1995)
result_n = reg_n.fit(cov_type='HC1')
print(result_n.summary())

                            OLS Regression Results                            
Dep. Variable:                lpackpc   R-squared:                       0.406
Model:                            OLS   Adj. R-squared:                  0.393
Method:                 Least Squares   F-statistic:                     38.86
Date:                Sun, 21 Mar 2021   Prob (F-statistic):           1.29e-07
Time:                        08:32:33   Log-Likelihood:                 12.724
No. Observations:                  48   AIC:                            -21.45
Df Residuals:                      46   BIC:                            -17.71
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.3389      0.935     11.060      0.0

### **Equation 12.11**
$\widehat{\text{ln}(Q^{cig}_i)} = 9.72 - 1.08 \text{ln}(P^{cig}_i).$

In [ ]:
reg_12_11 = iv.IV2SLS.from_formula(formula='lpackpc ~ 1 + [lravgprs ~ rtaxso]', data=cig_1995)
result_12_11 = reg_12_11.fit(cov_type='robust', debiased=True)
print(result_12_11.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                lpackpc   R-squared:                      0.4011
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3881
No. Observations:                  48   F-statistic:                    11.544
Date:                Sun, Mar 21 2021   P-value (F-stat)                0.0014
Time:                        08:53:04   Distribution:                  F(1,46)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      9.7199     1.5283     6.3598     0.0000      6.6435      12.796
lravgprs      -1.0836     0.3189    -3.3977     0.00

## **12.2 The General IV Regression Model**

### **Equation 12.15**
$\widehat{\text{ln}(Q^{cig}_i)} = 9.43 - 1.14 \text{ln}(P^{cig}_i) + 0.21 \text{ln}(Inc_i).$

In [ ]:
reg_12_15 = iv.IV2SLS.from_formula(formula='lpackpc ~ 1 + [lravgprs ~ rtaxso] + lperinc', data=cig_1995)
result_12_15 = reg_12_15.fit(cov_type='robust', debiased=True)
print(result_12_15.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                lpackpc   R-squared:                      0.4189
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3931
No. Observations:                  48   F-statistic:                    8.1911
Date:                Sun, Mar 21 2021   P-value (F-stat)                0.0009
Time:                        08:54:29   Distribution:                  F(2,45)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      9.4307     1.2594     7.4883     0.0000      6.8941      11.967
lperinc        0.2145     0.3117     0.6881     0.49

### **Equation 12.16**
$\widehat{\text{ln}(Q^{cig}_i)} = 9.89 - 1.28 \text{ln}(P^{cig}_i) + 0.28 \text{ln}(Inc_i).$

In [ ]:
reg_12_16 = iv.IV2SLS.from_formula(formula='lpackpc ~ 1 + [lravgprs ~ rtaxso + rtaxs] + lperinc', data=cig_1995)
result_12_16 = reg_12_16.fit(cov_type='robust', debiased=True)
print(result_12_16.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                lpackpc   R-squared:                      0.4294
Estimator:                    IV-2SLS   Adj. R-squared:                 0.4041
No. Observations:                  48   F-statistic:                    16.175
Date:                Sun, Mar 21 2021   P-value (F-stat)                0.0000
Time:                        08:57:46   Distribution:                  F(2,45)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      9.8950     0.9592     10.316     0.0000      7.9630      11.827
lperinc        0.2804     0.2539     1.1044     0.27

## **12.3 Checking Instrument Validity**